In [ ]:
import numpy as np
import pandas as pd
from numba import njit, prange

# グローバル変数の設定
state_list = np.array(['H', 'LR1', 'LR2', 'HR', 'DA', 'DAU', 'DB', 'DBU', 'DC', 'DCU', 'DD', 'DDU', 'D', 'PRH', 'PDAH', 'PDBH', 'PDCH', 'PDDH', 'PRL1', 'PRL2', 'PRLH'])
state_map = np.arange(len(state_list), dtype=np.int64)

# コストとユーティリティの設定
c_ma_f1_dukesA, c_ma_f1_dukesB, c_ma_f1_dukesC, c_ma_f1_dukesD = 1786482, 2056922, 2637803, 3179764
c_ma_f2_dukesA, c_ma_f2_dukesB, c_ma_f2_dukesC, c_ma_f2_dukesD = 34610, 34610, 43758, 2476258
c_fit, c_tcs, c_rem_lr_poly, c_rem_hr_poly = 2500, 15000, 100000, 150000
u_dukeA, u_dukeB, u_dukeC, u_dukeD, u_normal = 0.879, 0.879, 0.867, 0.844, 1.0

# 確率の設定
p_res_fit, p_speci_fit, p_res_tcs_afterFit = 0.40, 0.946, 0.40
p_s_fit_lowrisk1, p_s_fit_lowrisk2, p_s_fit_highrisk = 0.05, 0.10, 0.15
p_s_fit_dukeA, p_s_fit_dukeB, p_s_fit_dukeC, p_s_fit_dukeD = 0.20, 0.25, 0.30, 0.35
p_tcs_low1, p_tcs_low2, p_tcs_highrisk, p_s_tcs_DukeABCD = 0.10, 0.15, 0.20, 0.25
p_perforation_tcs, p_death_after_perforation, p_bleeding_with_resection = 0.0001, 0.05, 0.0001

@njit
def get_substate_probs(current_substate, p_res_fit, p_speci_fit, p_res_tcs_afterFit, p_s_fit_lowrisk1, p_s_fit_lowrisk2, p_s_fit_highrisk, p_s_fit_dukeA, p_s_fit_dukeB, p_s_fit_dukeC, p_s_fit_dukeD, p_tcs_low1, p_tcs_low2, p_tcs_highrisk, p_s_tcs_DukeABCD):
    if current_substate % 10 == 1:
        return np.array([p_res_fit, 1 - p_res_fit])
    elif current_substate % 100 == 11:
        state = current_substate // 100
        if state == 0: return np.array([p_speci_fit, 1 - p_speci_fit])
        elif state == 1: return np.array([1 - p_s_fit_lowrisk1, p_s_fit_lowrisk1])
        elif state == 2: return np.array([1 - p_s_fit_lowrisk2, p_s_fit_lowrisk2])
        elif state == 3: return np.array([1 - p_s_fit_highrisk, p_s_fit_highrisk])
        elif state == 4: return np.array([1 - p_s_fit_dukeA, p_s_fit_dukeA])
        elif state == 5: return np.array([1 - p_s_fit_dukeB, p_s_fit_dukeB])
        elif state == 6: return np.array([1 - p_s_fit_dukeC, p_s_fit_dukeC])
        elif state == 7: return np.array([1 - p_s_fit_dukeD, p_s_fit_dukeD])
    elif current_substate % 1000 == 111:
        return np.array([p_res_tcs_afterFit, 1 - p_res_tcs_afterFit])
    elif current_substate % 10000 == 1111:
        state = current_substate // 10000
        if state == 1: return np.array([1 - p_tcs_low1, p_tcs_low1])
        elif state == 2: return np.array([1 - p_tcs_low2, p_tcs_low2])
        elif state == 3: return np.array([1 - p_tcs_highrisk, p_tcs_highrisk])
        elif state in [4, 5, 6, 7]: return np.array([1 - p_s_tcs_DukeABCD, p_s_tcs_DukeABCD])
    return np.array([1.0])

@njit
def update_substate(current_substate, probs):
    if len(probs) == 1:
        return current_substate * 10 + 1
    else:
        return current_substate * 10 + (custom_choice(probs) + 1)

@njit
def get_next_substate(current_state, current_substate, cycle, last_tcs_cycle):
    if current_state == 0:
        if cycle - last_tcs_cycle <= 5: return 2
        elif cycle - last_tcs_cycle > 5: return 1
        else: return 3
    elif current_state in np.arange(1, 13) or current_state in np.array([13, 18, 19, 20]):
        return current_state * 10 + (2 if cycle - last_tcs_cycle <= 3 else 1)
    else:
        return current_substate

@njit
def Probs(M_it, stage, dur, change_probability, state_map, current_substate, p_death_after_perforation, p_perforation_tcs):
    n_s = len(state_map)
    v_p_it = np.zeros(n_s)
    
    if M_it == state_map[0]:
        if current_substate == 2:
            v_p_it[state_map[0]] = 1
        elif current_substate in np.array([1, 3]):
            v_p_it[state_map[0]] = 1 - change_probability[stage, 0]
            v_p_it[state_map[1]] = change_probability[stage, 0]
    elif M_it == state_map[1]:
        if current_substate == 12:
            v_p_it[state_map[18]] = 1
        elif current_substate in np.array([1112, 11112]):
            v_p_it[state_map[1]] = 1
        elif current_substate == 11111:
            v_p_it[state_map[12]] = p_death_after_perforation * p_perforation_tcs
            v_p_it[state_map[13]] = 1 - p_perforation_tcs
        else:
            v_p_it[state_map[1]] = 1 - change_probability[stage, 1]
            v_p_it[state_map[2]] = change_probability[stage, 1]
    # ... (他の状態についても同様に実装)

    assert np.isclose(v_p_it.sum(), 1, atol=1e-6)
    return v_p_it

@njit
def Costs(M_it, next_M_it, p_it, state_map, current_substate, c_fit, c_tcs, c_ma_f1_dukesA, c_ma_f1_dukesB, c_ma_f1_dukesC, c_ma_f1_dukesD, c_rem_lr_poly, c_rem_hr_poly, c_ma_f2_dukesA, c_ma_f2_dukesB, c_ma_f2_dukesC, c_ma_f2_dukesD):
    c_it = np.zeros_like(M_it, dtype=np.float64)
    
    for i in range(len(M_it)):
        if current_substate[i] % 1000 == 111:
            c_it[i] += c_fit
        if current_substate[i] % 100000 == 11121:
            c_it[i] += c_tcs
        
        if M_it[i] == state_map[4]:
            if next_M_it[i] == state_map[5] or (next_M_it[i] == state_map[12] and p_it[i, 5] > 0):
                c_it[i] += c_ma_f1_dukesA
        elif M_it[i] == state_map[6]:
            if next_M_it[i] == state_map[7] or (next_M_it[i] == state_map[12] and p_it[i, 7] > 0):
                c_it[i] += c_ma_f1_dukesB
        elif M_it[i] == state_map[8]:
            if next_M_it[i] == state_map[9] or (next_M_it[i] == state_map[12] and p_it[i, 9] > 0):
                c_it[i] += c_ma_f1_dukesC
        elif M_it[i] == state_map[10]:
            if next_M_it[i] == state_map[11] or (next_M_it[i] == state_map[12] and p_it[i, 11] > 0):
                c_it[i] += c_ma_f1_dukesD
        
        if current_substate[i] in np.array([11111, 181111, 191111]):
            c_it[i] += c_rem_lr_poly
        elif current_substate[i] in np.array([201111, 201111]):
            c_it[i] += c_rem_hr_poly
        
        if M_it[i] == state_map[5]:
            c_it[i] += c_ma_f2_dukesA
        elif M_it[i] == state_map[7]:
            c_it[i] += c_ma_f2_dukesB
        elif M_it[i] == state_map[9]:
            c_it[i] += c_ma_f2_dukesC
        elif M_it[i] == state_map[11]:
            c_it[i] += c_ma_f2_dukesD
    
    return c_it

@njit
def Effs(M_it, dur, state_map, current_substate, u_normal, u_dukeA, u_dukeB, u_dukeC, u_dukeD, X=None):
    if X is None:
        X = np.ones_like(M_it, dtype=np.float64)
    u_it = np.zeros_like(M_it, dtype=np.float64)
    
    normal_states = np.array([0, 1, 2, 3, 13, 18, 19, 20], dtype=np.int64)
    dukeA_states = np.array([4, 5], dtype=np.int64)
    dukeB_states = np.array([6, 7], dtype=np.int64)
    dukeC_states = np.array([8, 9], dtype=np.int64)
    dukeD_states = np.array([10, 11], dtype=np.int64)
    post_treatment_states = np.array([14, 15, 16, 17], dtype=np.int64)
    
    for i in range(len(M_it)):
        if custom_isin(np.array([M_it[i]], dtype=np.int64), state_map[normal_states])[0]:
            u_it[i] = u_normal
        elif custom_isin(np.array([M_it[i]], dtype=np.int64), state_map[dukeA_states])[0] or current_substate[i] in np.array([412, 422], dtype=np.int32):
            u_it[i] = u_dukeA
        elif custom_isin(np.array([M_it[i]], dtype=np.int64), state_map[dukeB_states])[0] or current_substate[i] in np.array([612, 622], dtype=np.int32):
            u_it[i] = u_dukeB
        elif custom_isin(np.array([M_it[i]], dtype=np.int64), state_map[dukeC_states])[0] or current_substate[i] in np.array([812, 822], dtype=np.int32):
            u_it[i] = u_dukeC
        elif custom_isin(np.array([M_it[i]], dtype=np.int64), state_map[dukeD_states])[0] or current_substate[i] in np.array([1012, 1022], dtype=np.int32):
            u_it[i] = u_dukeD
        elif M_it[i] == state_map[12]:
            u_it[i] = 0
        elif custom_isin(np.array([M_it[i]], dtype=np.int64), state_map[post_treatment_states])[0]:
            u_it[i] = u_normal
    
    return u_it * X


@njit
def custom_choice(probs):
    r = np.random.random()
    cumsum = np.cumsum(probs)
    return np.searchsorted(cumsum, r)

@njit
def custom_isin(arr, values):
    return np.array([x in values for x in arr], dtype=np.bool_)


@njit(parallel=True)
def MicroSim(v_M_1, n_i, n_t, states, change_probability, state_map, p_res_fit, p_speci_fit, p_res_tcs_afterFit, p_s_fit_lowrisk1, p_s_fit_lowrisk2, p_s_fit_highrisk, p_s_fit_dukeA, p_s_fit_dukeB, p_s_fit_dukeC, p_s_fit_dukeD, p_tcs_low1, p_tcs_low2, p_tcs_highrisk, p_s_tcs_DukeABCD, p_death_after_perforation, p_perforation_tcs, c_fit, c_tcs, c_ma_f1_dukesA, c_ma_f1_dukesB, c_ma_f1_dukesC, c_ma_f1_dukesD, c_rem_lr_poly, c_rem_hr_poly, c_ma_f2_dukesA, c_ma_f2_dukesB, c_ma_f2_dukesC, c_ma_f2_dukesD, u_normal, u_dukeA, u_dukeB, u_dukeC, u_dukeD, X=None, d_c=0.02, d_e=0.02, TR_out=True, TS_out=True, Trt=False, seed=1):
    np.random.seed(seed)
    v_dwc = 1 / (1 + d_c) ** np.arange(n_t + 1)
    v_dwe = 1 / (1 + d_e) ** np.arange(n_t + 1)
    m_M = np.empty((n_i, n_t + 1), dtype=np.int32)
    m_C = np.zeros((n_i, n_t + 1))
    m_E = np.zeros((n_i, n_t + 1))
    m_M[:, 0] = v_M_1
    dur = np.zeros(n_i, dtype=np.int64)
    last_tcs_cycle = np.full(n_i, -10, dtype=np.int32)
    current_substates = np.full(n_i, 1, dtype=np.int32)

    m_C[:, 0] = Costs(m_M[:, 0], m_M[:, 0], np.zeros((n_i, len(state_map))), state_map, current_substates, c_fit, c_tcs, c_ma_f1_dukesA, c_ma_f1_dukesB, c_ma_f1_dukesC, c_ma_f1_dukesD, c_rem_lr_poly, c_rem_hr_poly, c_ma_f2_dukesA, c_ma_f2_dukesB, c_ma_f2_dukesC, c_ma_f2_dukesD)
    m_E[:, 0] = Effs(m_M[:, 0], dur, state_map, current_substates, u_normal, u_dukeA, u_dukeB, u_dukeC, u_dukeD, X)
    
    for t in range(1, n_t + 1):
        m_p = np.zeros((n_i, len(state_map)))
        for i in prange(n_i):
            m_p[i] = Probs(m_M[i, t-1], t-1, dur[i], change_probability, state_map, current_substates[i], p_death_after_perforation, p_perforation_tcs)
        
        next_states = np.array([custom_choice(m_p[i]) for i in range(n_i)])
        
        for i in prange(n_i):
            current_substates[i] = get_next_substate(states[next_states[i]], current_substates[i], t, last_tcs_cycle[i])
            substate_probs = get_substate_probs(current_substates[i], p_res_fit, p_speci_fit, p_res_tcs_afterFit, p_s_fit_lowrisk1, p_s_fit_lowrisk2, p_s_fit_highrisk, p_s_fit_dukeA, p_s_fit_dukeB, p_s_fit_dukeC, p_s_fit_dukeD, p_tcs_low1, p_tcs_low2, p_tcs_highrisk, p_s_tcs_DukeABCD)
            current_substates[i] = update_substate(current_substates[i], substate_probs)
            
            if current_substates[i] % 100000 == 11121:
                last_tcs_cycle[i] = t

        m_C[:, t] = Costs(m_M[:, t - 1], next_states, m_p, state_map, current_substates, c_fit, c_tcs, c_ma_f1_dukesA, c_ma_f1_dukesB, c_ma_f1_dukesC, c_ma_f1_dukesD, c_rem_lr_poly, c_rem_hr_poly, c_ma_f2_dukesA, c_ma_f2_dukesB, c_ma_f2_dukesC, c_ma_f2_dukesD)
        m_M[:, t] = next_states
        m_E[:, t] = Effs(m_M[:, t], dur, state_map, current_substates, u_normal, u_dukeA, u_dukeB, u_dukeC, u_dukeD, X)
        dur_states = np.array([5, 7, 9, 11], dtype=np.int64)
        dur = np.where(custom_isin(m_M[:, t], state_map[dur_states]), dur + 1, 0)
        
    tc = m_C @ v_dwc
    te = m_E @ v_dwe
    tc_hat = np.mean(tc)
    te_hat = np.mean(te)
    
    results = {
        "m_M": m_M,
        "m_C": m_C,
        "m_E": m_E,
        "tc": tc,
        "te": te,
        "tc_hat": tc_hat,
        "te_hat": te_hat
    }
    return results

@njit
def post_process_results(m_M, m_C, m_E, tc, te, tc_hat, te_hat, n_i, n_t, state_map, TR_out=True, TS_out=True):
    results = {
        "tc": tc,
        "te": te,
        "tc_hat": tc_hat,
        "te_hat": te_hat
    }

    if TR_out:
        TR = np.zeros((n_t + 1, len(state_map)))
        for t in range(n_t + 1):
            for s in range(len(state_map)):
                TR[t, s] = np.sum(m_M[:, t] == state_map[s]) / n_i
        results["TR"] = TR

    if TS_out:
        TS = np.zeros((n_i, len(state_map)))
        for i in range(n_i):
            for s in range(len(state_map)):
                TS[i, s] = np.sum(m_M[i, :] == state_map[s]) / (n_t + 1)
        results["TS"] = TS

    return results

# メインの実行部分
if __name__ == "__main__":
    # パラメータの設定
    n_i = 10000  # シミュレーション対象の個人数
    n_t = 44  # シミュレーション期間（年数）
    v_M_1 = np.full(n_i, state_map[0], dtype=np.int32)  # 全員が健康状態から開始
    states = state_map.copy()

    # 遷移確率の読み込み
    change_probability = pd.read_csv('senni_pro.csv').set_index('time').to_numpy()

    # シミュレーションの実行
    raw_results = MicroSim(v_M_1, n_i, n_t, states, change_probability, state_map, 
                           p_res_fit, p_speci_fit, p_res_tcs_afterFit, 
                           p_s_fit_lowrisk1, p_s_fit_lowrisk2, p_s_fit_highrisk, 
                           p_s_fit_dukeA, p_s_fit_dukeB, p_s_fit_dukeC, p_s_fit_dukeD, 
                           p_tcs_low1, p_tcs_low2, p_tcs_highrisk, p_s_tcs_DukeABCD, 
                           p_death_after_perforation, p_perforation_tcs, 
                           c_fit, c_tcs, c_ma_f1_dukesA, c_ma_f1_dukesB, c_ma_f1_dukesC, c_ma_f1_dukesD, 
                           c_rem_lr_poly, c_rem_hr_poly, 
                           c_ma_f2_dukesA, c_ma_f2_dukesB, c_ma_f2_dukesC, c_ma_f2_dukesD, 
                           u_normal, u_dukeA, u_dukeB, u_dukeC, u_dukeD, 
                           X=None, d_c=0.02, d_e=0.02, TR_out=True, TS_out=True, 
                           Trt=False, seed=100)

    # 結果の後処理
    final_results = post_process_results(raw_results["m_M"], raw_results["m_C"], raw_results["m_E"],
                                         raw_results["tc"], raw_results["te"],
                                         raw_results["tc_hat"], raw_results["te_hat"],
                                         n_i, n_t, state_map, TR_out=True, TS_out=True)

    # 結果の表示
    print(f"平均費用: {final_results['tc_hat']:.2f}")
    print(f"平均効果: {final_results['te_hat']:.2f}")
    print(f"費用効果比: {final_results['tc_hat'] / final_results['te_hat']:.2f}")

    if 'TR' in final_results:
        print("\n各時点での状態分布:")
        print(final_results['TR'])

    if 'TS' in final_results:
        print("\n各個人の状態滞在時間の分布:")
        print(np.mean(final_results['TS'], axis=0))

RuntimeError: Specified LAPACK function could not be found.
